## Intro & the Dataset

I am including this notebook to briefly document the steps I took to turn a stack of FITS files taller than my house and fresh off the CCDs at the LCOGT into all the pretty diagrams that showed up on this poster.

<div>
<img src="images/poster.jpg" width="500" height="340">
</div>

The telescope images I worked with where taken of a young binary star by the name StHα-34. The images came in 3 separate color bands of visible light: U band, V band, and Z band (in order from shortest to longest wavelength). Each color band contained around 125 image sets, and each image set usually contained 3 images, for a total of just over 1000 total images, each contained in its own FITS file. Out of concern for over-exposure, images where taken in a short-exposure burst of 3 images. The script performs photometry on a median composite of each image set. For a better theoretical understanding of the project for which this package was developed, I have a shorter paper simplifying some of the theory of binary star accretion [here.](https://drive.google.com/file/d/1v-OLpeAwAxZ_o04SSG4qDhjf2u3Ahn1s/view?usp=sharing)


## Setting up the Environment

To set up all the dependencies I just need the requirements.txt file from the repo. Then from the shell I create and activate a virtual environment using the built in python module. Once inside the environment, I install all the packages I need from the text file.

In [ ]:
python -m venv 

./env/Scripts/activate

pip install -r requirements.txt

: 

## Running the Pipeline

With the environment all set up, I jump into a python shell and import the pipeline and start crunching through those FITS files.

In [ ]:
from psf_phot import pipeline
pipeline('../filenames.csv', '../fits/', '../results/')

: 

Since my stack of files was tall enough that I could use it to climb the Statue of Liberty, This took about 20 hours total, between all the starts and stops to fix random things. That was roughly 350 image sets, each containing 3 images, for a total of over 1000 FITS files total. After finishing, a run of a successful image set returned 4 files to the results folder:

- A new FITS file was created for the composite image of each set, also containing the targets metadata in a header
- A CSV file with all the tabulated data of the photometry results
- A PDF containing the images from the image set, the generated point spread function model, and optionally the residual images of the stars used to create the model; this was used diagnostically to make sure the code was working well
- A TXT file showing a bunch more diasgnostic info about how well the script ran for that image set.

Similar results are generated for image sets that failed, varying based on which point the image set failed.


## Creating the Light Curves

This is where the science starts and where the project actually gets interesting. I created light curves to track the brightness of the target star across the entire period of observation, and then ran a few different analyses on the cleaned data. This is also where the scope of this package ends. [Here](https://github.com/JeremyBuchanan/photometry-snippets.git) is a link to another repository, where I stashed all the other code I used to make my plots.

I used the ens_phot.py module from the second repo to build the light curves. Though the module contains several functions, I only needed two of them for my purposes: ens_match and ens_lc. First I needed to combine all the CSV files from all the image sets in the color band into a single file to be read into the matching script.

In [ ]:
cd results

cat *.csv > all.csv # combine the files

grep -v ra all.csv > all.csv # remove all the header rows

: 

Then give that CSV to the matching script.

In [ ]:
from ens_phot import ens_match, ens_lc
ens_match(in_file='all.csv', tra=73.59863195295, tdec=+17.16480415593, scope='lcogt', out_file='matched')

Ens_match searches through the data in 'all.csv' to pull out only the data for the stars at a set position (indicated by ra and dec). The output from ens_match is a 'pickle' file. It's basically a dictionary holding all the flux data of the target star. Then that file goes right into the light curve script.

In [ ]:
ens_lc(in_file='matched.p', target='target', date='4/21/22', scope='lcogt')

I used the ens_lc function to actually generate the light curves for the project. The script brings up an interactive window with tons of functionality to manipulate the data. It is a little tricky to get the hang of, but it works really well, and the commands are explaned well in the documentation of the function. The idea is to clean the data of the target star to yield a very uniform light curve. Two main steps were used to clean the light curves.

1. I selected for the most uniform images by trimming off the very bright and very dim images of the target. The justification is that these images are likely not accurate representations of the actual target, because they are polluted either by cloud cover or moonlight.
2. Even among the main segment of the image sets, there are outlier image sets. In short terms, changes to the target's brightness are tracked by comparing how the target appears in context of the stars around it. Occasionally, one of the stars near the target will appear much brighter, perhaps due to a hot pixel on the telescope chip, or another obstruction. The issue can be solved by removing the very bright (or very dim) star from the context of the target, or in extreme cases by removing the bad image set entirely.

The output of ens_lc is a DAT file containing the light curve, which I used to finally plot everything.


## Plotting Everything

The first plots are the light curves. I'm not going to be concerned with pulling any conclusions out of the plots here; I'll save that for an actual published paper.

In [ ]:
python lc.py


<div>
<img src="images/periods.png" width="500" height="340">
</div>

<div>
<img src="images/phasefold.png" width="500" height="340">
</div>

The first plot is nothing special; its just the light curve itself. The second plot, on the other hand, is called a "phase folded light curve." By knowing the initial start date of observation and the length of the orbital period of the binary, I was able to plot the data from each period of orbit on top of each other, making it easier to identify patterns throughout the orbit of the binary. Again, the conclusions are not really important, just the process for running all these scripts to recreate the results.


Next up to be plotted is the periodogram. This is a pretty interesting one, and I think it's worthwhile to dig into the math a bit.

In [ ]:
python p_gram.py

<div>
<img src="images/periodogram.png" width="500" height="300">
</div>

I plotted what is known as a Lomb-Scargle periodogram. Similar to a fast Fourier transform, this periodogram is better suited to datasets with data points at less consistent time intervals. Other than that, it is a similar technique for breaking the variations in the data into a sum of simple sine waves of varying frequencies. The periodogram shows the "power" (essentially the degree to which there is periodic movement) over the period of the binary's orbit, which can show whether or not the data changes in sync with the period of orbit.


Finally is the color analysis plot. While the other plots contained the data from 3 separate color bands, only the U band, or the "bluest" light from the target was studied in this plot.

In [ ]:
python color_analysis.py

<div>
<img src="images/colorplots.png" width="500" height="340">
</div>

This plot is fairly straight forward. It shows how the color of the target changes as the intensity of the targets light changes. Without getting into too many specifics, it show that when the light from the target appears brighter, it also appears to be "bluer" light than normal.


## Closing

My intent with this walkthrough is to show one example for how this code could be taken from raw telescope data all the way to a couple presentable plots. I also wanted to highlight a few of the tools used for analyzing all the data processed by this package, to show -by adding some context- a practical use case for this script. 